In [11]:
!pip install numpy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

image = cv2.imread('data\discharge_print_128x128.png')

# cv2 opens in BGR format for some reason
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Recommended to convert to float32 do the mathematical expressions with more precision
im = image_rgb.astype(np.float32)

print(f"Image shape: {im.shape}")

plt.imshow(im.astype(np.uint8))
plt.title("Original Image")
plt.axis('off')
plt.show()

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\guief\AppData\Local\Temp\ipykernel_33240\4182394453.py:5: SyntaxWarning: invalid escape sequence '\d'
  image = cv2.imread('data\discharge_print_128x128.png')
C:\Users\guief\AppData\Local\Temp\ipykernel_33240\4182394453.py:5: SyntaxWarning: invalid escape sequence '\d'
  image = cv2.imread('data\discharge_print_128x128.png')


ModuleNotFoundError: No module named 'numpy'

In [4]:
def generate_zero_padding_kernel(im, k):
    pad_width = k // 2

    if im.ndim == 3:
        m, n, c = im.shape
        im_pad = np.zeros((m + 2 * pad_width, n + 2 * pad_width, c), dtype=im.dtype)
        im_pad[pad_width:-pad_width, pad_width:-pad_width, :] = im
    elif im.ndim == 2:
        m, n = im.shape
        im_pad = np.zeros((m + 2 * pad_width, n + 2 * pad_width), dtype=im.dtype)
        im_pad[pad_width:-pad_width, pad_width:-pad_width] = im
    else:
        raise ValueError("Image must have either 2 or 3 dimensions")

    return im_pad

def generate_zero_padding_fft(im):
    m, n, c = im.shape
    im_pad = np.zeros((2*m, 2*n, c))
    im_pad[:m, :n, :] = im
    print(im.shape, im_pad.shape)
    return im_pad

def generate_gausian_matrix(shape):
    gaus = np.random.randn(*shape)

    return gaus

# This returns a mean vector (one mean for each channel)
u = np.mean(im, axis=(0, 1))


tu = (im - u)/np.sqrt(im[:,:,0].size)

tu_pad = generate_zero_padding_fft(tu)

W = generate_gausian_matrix(tu_pad.shape)

tu_fft = np.fft.fft2(tu_pad, axes=(0, 1))
W_fft = np.fft.fft2(W, axes=(0, 1))

convolution = tu_fft * W_fft

im_convoluted = np.fft.ifft2(convolution, axes=(0, 1))

# Remove padding
im_convoluted_unpad = im_convoluted[0:im.shape[0], 0:im.shape[1]]

# Remember to get that ifft returns a complex number!
result = u + np.real(im_convoluted_unpad)

# The previous operations make it so that result has negative values, we need to clip them
final_result = np.clip(result, 0, 255).astype(np.uint8)

plt.imshow(final_result)

NameError: name 'np' is not defined

Problems:

- Generated three different gaussian noises, one for each color. Since they were randomized separately, this messed up the covariance between colors. To fix this i should have done one random and copy this to the other 2 channels -> Explained in the 3.1 section of the paper


- Maybe the convolution was wrong. I was not supposed to multiply the magnitude of the image and the gaussian noise's magnitude? Maybe preserve the magnitude from the original image and then just replace the phase by the gaussian's, proving that for textures that does not change anything.




Fix:
Generate three equal noises

Change the logic to:

U = v + F-1( |t_v|  /_ arg(W)+arg(W)  )

Basically we maintain the magnitude from our "bord" (the ohmega set) and randomize the phase, making it chaotic, as a microtexture should be.



OBS: Since im doing "manually", there will be no need to do the padding

In [5]:
def generate_zero_padding_kernel(im, k):
    pad_width = k // 2

    if im.ndim == 3:
        m, n, c = im.shape
        im_pad = np.zeros((m + 2 * pad_width, n + 2 * pad_width, c), dtype=im.dtype)
        im_pad[pad_width:-pad_width, pad_width:-pad_width, :] = im
    elif im.ndim == 2:
        m, n = im.shape
        im_pad = np.zeros((m + 2 * pad_width, n + 2 * pad_width), dtype=im.dtype)
        im_pad[pad_width:-pad_width, pad_width:-pad_width] = im
    else:
        raise ValueError("Image must have either 2 or 3 dimensions")

    return im_pad

def generate_zero_padding_fft(im):
    m, n, c = im.shape
    im_pad = np.zeros((2*m, 2*n, c))
    im_pad[:m, :n, :] = im
    print(im.shape, im_pad.shape)
    return im_pad

def generate_gausian_matrix(shape):
    gaus = np.random.randn(*shape)

    return gaus

# This returns a mean vector (one mean for each channel)
u = np.mean(im, axis=(0, 1))


tu = (im - u)


single_channel_noise = np.array(generate_gausian_matrix(tu.shape[0:2]))

W = np.dstack([single_channel_noise, single_channel_noise, single_channel_noise])

tu_fft = np.fft.fft2(tu, axes=(0, 1))
tu_mag = np.abs(tu_fft)
tu_phase = np.angle(tu_fft)


W_fft = np.fft.fft2(W, axes=(0, 1))
W_mag = np.abs(W_fft)
W_phase = np.angle(W_fft)

# Assemble both
result = tu_mag * np.exp(1j * W_phase)

image_gaussian_noise = np.fft.ifft2(result, axes=(0, 1))


# Remember to get that ifft returns a complex number!
result = u + np.real(image_gaussian_noise)

# The previous operations make it so that result has negative values, we need to clip them
final_result = np.clip(result, 0, 255).astype(np.uint8)

plt.imshow(final_result)

NameError: name 'np' is not defined

In [6]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5)) # Adicionar figsize é uma boa prática


ax[0].imshow(im.astype(np.uint8))
ax[0].set_title("Original Image")

ax[1].imshow(final_result)
ax[1].set_title("Synthesized Result")

ax[0].axis('off')
ax[1].axis('off')

plt.tight_layout()

plt.show()

NameError: name 'plt' is not defined

TO DO: add the masks to see the results